<h3>Глобальные переменные</h3>

In [1]:
transformer_model_path = 'DeepPavlov/rubert-base-cased'#'sagteam/xlm-roberta-large-sag'
USE_CUDA = True
ADD_CONTEXT = False
USE_CYCLR = False

<h3>Векторизуем словарь Meddra</h3>

In [2]:
from vectorization import ConceptVectorizer

In [3]:
#альтернатива
CV = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=False, use_model=False, use_cuda=USE_CUDA)
CV_with_conceptless = ConceptVectorizer(transformer_model_path, '../../Data/External/pt_rus.asc', \
                             use_concept_less=True, use_model=False, use_cuda=USE_CUDA)

Загрузка готовых вложений (если они есть). Загружать вложения после fit_transform() не надо

In [4]:
import torch

CV.thesaurus_embeddings = torch.load('./Model_weights/rubert_thesaurus_embeddings_meddra_origin.pt')
CV.normalization_mode = 'mean_pooling'

<h3>Обработка датасета RDRS</h3>

In [10]:
import jsonlines
import numpy as np
from sklearn.model_selection import train_test_split
from tools.parse_RDRS import *
import pandas as pd

In [6]:
ds = []
with jsonlines.open('../../Data/Raw/medNorm_16022022.jsonlines') as reader:
    for obj in reader:
        ds.append(obj)
        
X_train, X_test = train_test_split(ds, test_size=0.33, random_state=42)

In [7]:
ds_parsed, errors = parse_json_ds(X_train=X_train, \
                                  X_test=X_test, \
                                  CV=CV, \
                                  CV_with_conceptless = CV_with_conceptless, \
                                  add_context=ADD_CONTEXT)

test_phrases, test_concepts = ds_parsed['test']['phrases'], ds_parsed['test']['concepts']

In [15]:
mh_rus = MeddraHelper(meddra_path_pt = '../../Data/External/pt_rus.asc', 
                      meddra_path_llt = '../../Data/External/llt_rus.asc',
                      lang = 'RUS', 
                      umls_path = '../../Data/External/MRCONSO.RRF')

<h2>Обучение на llt концептах</h2>

In [11]:
df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_hlt_corp_with_terms.csv')

In [12]:
df = df[df['corp_name']=='meddra_llt']

In [13]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa
import pickle
import os

#Гиперпараметры
batch_size=64
epochs = 30

all_splits_score_dfs = []

input_format = 'sentences' if ADD_CONTEXT else 'phrases'

torch.manual_seed(0)
np.random.seed(0)

#фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
train_phrases, train_concepts = df['phrase'].to_list() , df['pt code'].apply(lambda x: str(x)).to_list() 

if not ADD_CONTEXT:
    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
else:
    ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)

#if os.exists('./Model_weights/ds_train.pickl'):
#    with open('./Model_weights/ds_train.pickl', 'r') as f:
#        ds_train = pickle.load(f)
#else:
#    if not ADD_CONTEXT:
#        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
#    else:
#        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        
net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
#net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
if USE_CYCLR:
    scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0002, cycle_momentum=False)
scaler = torch.cuda.amp.GradScaler()
#early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

#valid_evaluator = Evaluator(train_concepts, val_concepts)
test_evaluator = Evaluator(train_concepts, test_concepts)

initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                if USE_CYCLR:
                    scheduler.step()
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                if USE_CYCLR:
                    scheduler.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

    test_evaluator.get_all_f1_scores(model_answers)
    #if epoch==30:
    #    torch.save(net, './Model_weights/rubert_only_additional_corp_with_llt_%sep'%epoch)
        
torch.save(net, './Model_weights/rubert_only_additional_corp_with_llt_%sep'%epoch)
torch.save(optimizer, './Model_weights/opt_rubert_only_additional_corp_with_llt_%sep'%epoch)
print('On %sep:'%epoch)
test_evaluator.get_all_f1_scores(model_answers)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSe

All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.106945  0.042991  0.106973  0.043035        0.0      0.0

100%|██████████| 3787/3787 [00:40<00:00, 93.67batch/s] 


All             In VOC           Out of VOC         
       f1 micro  f1 macro f1 micro  f1 macro   f1 micro f1 macro
score  0.125957  0.048313  0.12599  0.048362        0.0      0.0

100%|██████████| 3787/3787 [00:38<00:00, 97.46batch/s] 


...

All             In VOC           Out of VOC         
       f1 micro f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.310272  0.10591  0.310354  0.106105        0.0      0.0

100%|██████████| 3787/3787 [00:39<00:00, 94.73batch/s] 


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.332981  0.106862  0.333069  0.107054        0.0      0.0

On 30ep:


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.332981  0.106862  0.333069  0.107054        0.0      0.0

<h2>Обучение на переведенном корпусе + hlt</h2>

In [16]:
df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_hlt_corp_with_terms.csv')
df = df[df['corp_name']!='meddra_llt']

In [17]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa
import pickle
import os

#Гиперпараметры
batch_size=64
epochs = 30

all_splits_score_dfs = []

input_format = 'sentences' if ADD_CONTEXT else 'phrases'

torch.manual_seed(0)
np.random.seed(0)

#фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
train_phrases, train_concepts = df['phrase'].to_list() , df['pt code'].apply(lambda x: str(x)).to_list() 

if not ADD_CONTEXT:
    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
else:
    ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)

#if os.exists('./Model_weights/ds_train.pickl'):
#    with open('./Model_weights/ds_train.pickl', 'r') as f:
#        ds_train = pickle.load(f)
#else:
#    if not ADD_CONTEXT:
#        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
#    else:
#        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        
net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
#net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
if USE_CYCLR:
    scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0002, cycle_momentum=False)
scaler = torch.cuda.amp.GradScaler()
#early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

#valid_evaluator = Evaluator(train_concepts, val_concepts)
test_evaluator = Evaluator(train_concepts, test_concepts)

initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                if USE_CYCLR:
                    scheduler.step()
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                if USE_CYCLR:
                    scheduler.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

    test_evaluator.get_all_f1_scores(model_answers)
    #if epoch==30:
    #    torch.save(net, './Model_weights/rubert_only_additional_corp_with_llt_%sep'%epoch)
        
torch.save(net, './Model_weights/rubert_only_additional_corp_with_hlt_googletrans_%sep'%epoch)
torch.save(optimizer, './Model_weights/opt_rubert_only_additional_corp_with_hlt_googletrans_%sep'%epoch)
print('On %sep:'%epoch)
test_evaluator.get_all_f1_scores(model_answers)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 3787/3787 [00:43<00:00, 87.72batch/s] 


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.135992  0.037086  0.143504  0.038439   0.031496  0.024872

100%|██████████| 3787/3787 [00:43<00:00, 86.97batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.173752  0.054095  0.184263  0.061434   0.027559  0.020775

100%|██████████| 3787/3787 [00:40<00:00, 92.60batch/s] 


...

All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.398204  0.080916  0.425984  0.098873   0.011811  0.007527

100%|██████████| 3787/3787 [00:39<00:00, 94.89batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.421178  0.092407  0.450609  0.113575   0.011811  0.006453

On 30ep:


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.421178  0.092407  0.450609  0.113575   0.011811  0.006453

<h2>Обучение на доп. корпусе v3</h2>

Он содержит всю переведенную часть + hlt + llt

In [9]:
import pandas as pd

In [10]:
df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_hlt_corp_with_terms.csv')
df

,phrase,pt term,pt code,corp_name,lang,phrase_translated
0,боль в суставах,Артралгия,10003239,CADEC,en,pain in joints\n
1,боль в моей левой ноге,Боль в конечности,10033425,CADEC,en,pain in my left leg\n
2,мышечные проблемы,Миопатия,10028641,CADEC,en,muscle problems\n
3,мышечные боли,Миалгия,10028411,CADEC,en,muscle aches\n
4,Сильная интенсивная левая рука и боль в плече,Боль,10033371,CADEC,en,severe intense left arm and shoulder pain\n
...,...,...,...,...,...,...
99525,Генетические полиморфизмы,Полиморфизм CYP3A5,10083372,meddra_hlt,ru,NaN
99526,Коронавирусные инфекции,Постострый синдром COVID-19,10085503,meddra_hlt,ru,NaN
99527,"Инфекции, обусловленные вирусом гепатита",Врожденный гепатит C,10084252,meddra_hlt,ru,NaN
99528,Диагностические процедуры для оценки нарушений...,Тест импульсного движения головы,10086191,meddra_hlt,ru,NaN


In [13]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa
import pickle
import os

#Гиперпараметры
batch_size=64
epochs = 30

all_splits_score_dfs = []

input_format = 'sentences' if ADD_CONTEXT else 'phrases'

torch.manual_seed(0)
np.random.seed(0)

#фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
train_phrases, train_concepts = df['phrase'].to_list() , df['pt code'].apply(lambda x: str(x)).to_list() 

if not ADD_CONTEXT:
    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
else:
    ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)

#if os.exists('./Model_weights/ds_train.pickl'):
#    with open('./Model_weights/ds_train.pickl', 'r') as f:
#        ds_train = pickle.load(f)
#else:
#    if not ADD_CONTEXT:
#        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
#    else:
#        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        
net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
#net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
if USE_CYCLR:
    scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0002, cycle_momentum=False)
scaler = torch.cuda.amp.GradScaler()
#early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

#valid_evaluator = Evaluator(train_concepts, val_concepts)
test_evaluator = Evaluator(train_concepts, test_concepts)

initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                if USE_CYCLR:
                    scheduler.step()
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                if USE_CYCLR:
                    scheduler.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

    test_evaluator.get_all_f1_scores(model_answers)
    if epoch==30:
        net.save('./Model_weights/rubert_only_additional_corp_v3_%sep'%epoch)
        
net.save('./Model_weights/rubert_only_additional_corp_v3_%sep'%epoch)
torch.save(optimizer, './Model_weights/rubert_only_additional_corp_v3_%sep'%epoch)
print('On %sep:'%epoch)
test_evaluator.get_all_f1_scores(model_answers)

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 3787/3787 [00:40<00:00, 92.54batch/s] 


All             In VOC          Out of VOC         
       f1 micro f1 macro  f1 micro f1 macro   f1 micro f1 macro
score  0.141009  0.05032  0.141009  0.05032        0.0      0.0

100%|██████████| 3787/3787 [00:40<00:00, 94.58batch/s] 


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.197518  0.062382  0.197518  0.062382        0.0      0.0

100%|██████████| 3787/3787 [00:38<00:00, 97.58batch/s] 


...

100%|██████████| 3787/3787 [00:39<00:00, 96.15batch/s] 


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.453393  0.110816  0.453393  0.110816        0.0      0.0

100%|██████████| 3787/3787 [00:39<00:00, 96.17batch/s] 


All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.456826  0.124116  0.456826  0.124116        0.0      0.0

AttributeError: 'CADEC_SoTa' object has no attribute 'save'

In [15]:
torch.save(net, './Model_weights/rubert_only_additional_corp_v3_%sep'%epoch)

---

<h2>Кроссвалидация дообученного rubert без cyclic lr</h2>

In [16]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import *
from models import CADEC_SoTa

#Гиперпараметры
batch_size=16
epochs = 50

all_splits_score_dfs = []

input_format = 'sentences' if ADD_CONTEXT else 'phrases'

for fold_num in range(1, n_splits+1):
    print('Сплит %s'%fold_num)
    torch.manual_seed(0)
    np.random.seed(0)
    best_model_path = './Model_weights/rubert_additionalcorp_no_cyclic_lr_es_split_%s.pt'%fold_num
    
    #фразы могут быть с контекстом или без контекста, в зависимости от флага ADD_CONTEXT
    train_phrases, train_concepts = all_splits[fold_num]['train'][input_format], all_splits[fold_num]['train']['concepts']
    val_phrases, val_concepts = all_splits[fold_num]['validation'][input_format], all_splits[fold_num]['validation']['concepts']
    test_phrases, test_concepts = all_splits[fold_num]['test'][input_format], all_splits[fold_num]['test']['concepts']
    
    if not ADD_CONTEXT:
        ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
    else:
        ds_train = MedNormContextDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
        ds_test = MedNormContextDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
        ds_val = MedNormContextDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)
        
    #net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
    #перемещение модели на вычислительные мощности, либо cpu, либо cuda
    net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')
    
    device = 'cuda' if USE_CUDA else 'cpu'
    net.to(device)
    
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.00002) #, cycle_momentum=True
    if USE_CYCLR:
        scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0002, cycle_momentum=False)
    scaler = torch.cuda.amp.GradScaler()
    early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                              shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
    valloader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False)

    valid_evaluator = Evaluator(train_concepts, val_concepts)
    test_evaluator = Evaluator(train_concepts, test_concepts)

    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA:
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    if USE_CYCLR:
                        scheduler.step()
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    if USE_CYCLR:
                        scheduler.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        with tqdm(valloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

                model_answers.append(pred_meddra_code)

        valid_evaluator.get_all_f1_scores(model_answers)
        early_stopping(valid_evaluator.f1_scores_df['All']['f1 macro']['score'], net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    net.load_state_dict(torch.load(best_model_path))



    print('Evaluating on test dataset')
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

        test_evaluator.get_all_f1_scores(model_answers)
    all_splits_score_dfs.append(test_evaluator.f1_scores_df)

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df

Сплит 1


100%|██████████| 1503/1503 [00:19<00:00, 78.88batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.688623  0.295189  0.715693  0.337163   0.219512  0.111751

Validation metric increased (-inf --> 0.295189).  Saving model ...


100%|██████████| 1503/1503 [00:17<00:00, 83.61batch/s]


...

All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.757812  0.351142  0.796547  0.436114   0.078947  0.040984

EarlyStopping counter: 7 out of 7
Early stopping
Evaluating on test dataset


100%|██████████| 2469/2469 [00:28<00:00, 85.18batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.756177  0.361156  0.792453  0.476983   0.138686  0.064251

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score   0.76079  0.333561  0.794298  0.406698    0.17519   0.09817
std         0.007262  0.022379  0.007151  0.041383   0.026218  0.020097

<h2>Кроссвалидация без контекста</h2>

In [7]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np
from tools.pytorchtools import EarlyStopping
import pandas as pd
from torch.optim.lr_scheduler import CyclicLR
from dataset import MedNormDataset


#Гиперпараметры
batch_size=16
epochs = 50

all_splits_score_dfs = []

for fold_num in range(1, n_splits+1):
    print('Сплит %s'%fold_num)
    best_model_path = './Model_weights/rubert_cyclic_lr_es_split_%s.pt'%fold_num
    
    train_phrases, train_concepts = all_splits[fold_num]['train']['phrases'], all_splits[fold_num]['train']['concepts']
    val_phrases, val_concepts = all_splits[fold_num]['validation']['phrases'], all_splits[fold_num]['validation']['concepts']
    test_phrases, test_concepts = all_splits[fold_num]['test']['phrases'], all_splits[fold_num]['test']['concepts']
    
    ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
    ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
    ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)

    net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
    scheduler = CyclicLR(optimizer, base_lr=0.00002, max_lr=0.0001, cycle_momentum=False)
    scaler = torch.cuda.amp.GradScaler()
    early_stopping = EarlyStopping(path=best_model_path, mode='max', verbose=True)

    trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                              shuffle=False, num_workers=0)
    testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
    valloader = torch.utils.data.DataLoader(ds_val, batch_size=1, shuffle=False)

    valid_evaluator = Evaluator(train_concepts, val_concepts)
    test_evaluator = Evaluator(train_concepts, test_concepts)

    initial_loss = None

    for epoch in range(1, epochs+1):
        #обучение модели в эпохе
        net.train()
        with tqdm(trainloader, unit="batch") as tepoch:
            for data in tepoch:

                tepoch.set_description(f"Epoch {epoch}")

                inputs = data['tokenized_phrases']
                labels = data['one_hot_labels']

                optimizer.zero_grad()
                if USE_CUDA=='cuda':
                    with torch.cuda.amp.autocast():
                        outputs = net(inputs)['output']
                        loss = criterion(outputs, labels)
                    scaler.scale(loss).backward()
                    scaler.step(optimizer)
                    scheduler.step()
                    scaler.update()
                else:
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                    loss.backward()
                    optimizer.step()
                    scheduler.step()
                if initial_loss is None:
                    initial_loss = loss.item()
                tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
        #расчет точностей модели после эпохи
        net.eval()
        model_answers=[]
        with tqdm(valloader, unit="batch") as eval_process:
            for data in eval_process:

                inputs = data['tokenized_phrases']

                with torch.no_grad():
                    outputs_dict = net(inputs)
                    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

                model_answers.append(pred_meddra_code)

        valid_evaluator.get_all_f1_scores(model_answers)
        early_stopping(valid_evaluator.f1_scores_df['All']['f1 macro']['score'], net)
        if early_stopping.early_stop:
            print("Early stopping")
            break
    net.load_state_dict(torch.load(best_model_path))



    print('Evaluating on test dataset')
    net.eval()
    model_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

            model_answers.append(pred_meddra_code)

        test_evaluator.get_all_f1_scores(model_answers)
    all_splits_score_dfs.append(test_evaluator.f1_scores_df)

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])
f1_mean_scores_df.loc['std', :] = np.std(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array(all_splits_score_dfs), axis = 0)
f1_mean_scores_df

Сплит 1


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.
100%|██████████| 1503/1503 [00:17<00:00, 85.55batch/s]


All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.695941  0.303515  0.724842  0.357223   0.195122  0.093363

Validation metric increased (-inf --> 0.303515).  Saving model ...


100%|██████████| 1503/1503 [00:17<00:00, 88.26batch/s] 


...

All             In VOC           Out of VOC          
       f1 micro f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.745028  0.31258  0.780781  0.380135   0.118421  0.048009

EarlyStopping counter: 7 out of 7
Early stopping
Evaluating on test dataset


100%|██████████| 2469/2469 [00:25<00:00, 97.17batch/s] 


All              In VOC           Out of VOC          
      f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.73471  0.309099  0.770154  0.402704   0.131387  0.059365

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score  0.761452  0.326338  0.797758  0.411703   0.127511  0.071472
std         0.013717  0.013684   0.01462  0.022687   0.015239   0.01367

In [12]:
from models import CADEC_SoTa


net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)
print('Net loaded')
net.load_state_dict(torch.load(best_model_path))

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


<All keys matched successfully>

In [ ]:
model_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]

        model_answers.append(pred_meddra_code)
            
    test_evaluator.get_all_f1_scores(model_answers)

In [ ]:
from sklearn.model_selection import KFold

kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(ds):
    train_reviews = np.array(ds)[train_index]
    test_reviews = np.array(ds)[test_index]

In [9]:
from evaluator import Evaluator

evaluator = Evaluator(train_concepts, test_concepts)

---

<h4>Добавим дополнительную часть</h4>

In [11]:
import pandas as pd

#df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_corp.csv')
df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_hlt_corp.csv')
train_concepts = train_concepts + df['pt code'].apply(lambda x: str(x)).to_list()
train_phrases = train_phrases + df['phrase'].to_list()

---

<h4>Чисто доп. часть подгружаем</h4>

In [11]:
import pandas as pd
from dataset import MedNormDataset

df = pd.read_csv('../../Data/CorpsStatisticData/FULL_prepared_corp.csv')
train_concepts = df['pt code'].apply(lambda x: str(x)).to_list()
train_phrases = df['phrase'].to_list()

#ds_train = MedNormDataset(train_phrases, train_concepts, CV_train, use_cuda=USE_CUDA)
#ds_test = MedNormDataset(train_phrases, train_concepts, CV_train, use_cuda=USE_CUDA)

In [24]:
from dataset import MedNormDataset

ds_train = MedNormDataset(train_phrases, train_concepts, CV, use_cuda=USE_CUDA)
ds_test = MedNormDataset(test_phrases, test_concepts, CV, use_cuda=USE_CUDA)
ds_val = MedNormDataset(val_phrases, val_concepts, CV, use_cuda=USE_CUDA)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


<h2>Обучение</h2>

<h4>Настройка глобальных переменных для большей детерменированности</h4>

In [27]:
import os 

torch.manual_seed(0)
np.random.seed(0)

<h4>Импорт модели</h4>

In [28]:
from models import CADEC_SoTa


net = CADEC_SoTa(transformer_model_path, CV.thesaurus_embeddings)
#перемещение модели на вычислительные мощности, либо cpu, либо cuda
device = 'cuda' if USE_CUDA else 'cpu'
net.to(device)
print('Net loaded')

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Net loaded


In [33]:
#или
import re
net = torch.load('./Model_weights/rubert_only_additional_corp_v2_30ep.pt')

#замарозка весов
num_layers = [re.sub('\D', '', name) for name, _ in net.named_parameters() if re.search('\d', name)][-1]
for name, param in net.named_parameters():
    if name.find('encoder')>=0:
        if name.find('layer.'+str(num_layers))<0:
            param.requires_grad=False   
    elif name.find('embeddings')>=0:
        param.requires_grad=False

<h4>Инициализация доп. объектов для обучения</h4>

In [30]:
import torch.optim as optim
import torch.nn as nn
import torch
import numpy as np

from torch.optim.lr_scheduler import CyclicLR
    
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(net.parameters(), lr=0.0001) #, cycle_momentum=True
#scheduler = CyclicLR(optimizer, base_lr=4.7e-05, max_lr=0.0002, cycle_momentum=False)
scaler = torch.cuda.amp.GradScaler()

<h4>Цикл обучения</h4>

In [35]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score
from evaluator import Evaluator
import numpy as np



#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)


initial_loss = None

for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'][0])
    evaluator.get_all_f1_scores(real_answers, model_answers)
print('Finished Training')
torch.save(net,'./Model_weights/rubert_additional_corp_freeze_layers_%s_exp_17.pt'%epoch)

100%|██████████| 3787/3787 [00:39<00:00, 95.63batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.597043  0.177648  0.621208  0.199196   0.149485  0.090458

100%|██████████| 3787/3787 [00:39<00:00, 95.16batch/s] 


All              In VOC           Out of VOC          
   f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
0  0.621336  0.187186  0.647092  0.215558    0.14433  0.087673

100%|██████████| 3787/3787 [00:38<00:00, 97.60batch/s] 


...

100%|██████████| 3787/3787 [00:38<00:00, 97.19batch/s] 


All             In VOC           Out of VOC          
   f1 micro  f1 macro f1 micro  f1 macro   f1 micro  f1 macro
0  0.733034  0.267256  0.76816  0.345812   0.082474  0.053866

100%|██████████| 3787/3787 [00:40<00:00, 94.30batch/s] 


All              In VOC          Out of VOC          
   f1 micro  f1 macro  f1 micro f1 macro   f1 micro  f1 macro
0  0.736731  0.270444  0.772613  0.35222   0.072165  0.050562

Finished Training


<h3>Анализ ошибок RuBERT</h3>

In [108]:
import pandas as pd

model_df_out = pd.DataFrame({'true': model_answers, 'pred': real_answers, 'phrases': phrases, 'test_ids': test_ids})
model_df_out['true_terms'] = model_df_out['true'].apply(lambda x: CV_train.meddra_code_to_meddra_term[x])
model_df_out['pred_terms'] = model_df_out['pred'].apply(lambda x: CV_train.meddra_code_to_meddra_term[x])
model_df_out['is wrong'] = model_df_out['pred'] != model_df_out['true']
model_df_out[model_df_out['is wrong']].to_csv('rubert_50_epochs_out.csv', index=False)

---

<h2>Черновой код</h2>

In [19]:
import pandas as pd

columns_tuples = [('All', 'f1 micro'), 
                 ('All', 'f1 macro'),

                 ('In VOC', 'f1 micro'), 
                 ('In VOC', 'f1 macro'),

                 ('Out of VOC', 'f1 micro'), 
                 ('Out of VOC', 'f1 macro')]

all_f1_scores = np.random.uniform(low=0.0, high=1.0, size=(1, 6))
col_index = pd.MultiIndex.from_tuples(columns_tuples)
f1_scores_df = pd.DataFrame(all_f1_scores, columns=col_index, index=['score'])
display(f1_scores_df)
all_f1_scores = np.random.uniform(low=0.0, high=1.0, size=(1, 6))
f1_scores_df_2 = pd.DataFrame(all_f1_scores, columns=col_index, index=['score'])
display(f1_scores_df_2)

f1_mean_scores_df = pd.DataFrame(columns=col_index, index=['mean score', 'std'])

f1_mean_scores_df.loc['std', :] = np.std(np.array([f1_scores_df, f1_scores_df_2]), axis = 0)
f1_mean_scores_df.loc['mean score', :] = np.mean(np.array([f1_scores_df, f1_scores_df_2]), axis = 0)
f1_mean_scores_df

All              In VOC           Out of VOC          
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
score  0.425354  0.454538  0.419959  0.399042   0.982587  0.465343

All              In VOC           Out of VOC         
       f1 micro  f1 macro  f1 micro  f1 macro   f1 micro f1 macro
score  0.914319  0.578006  0.813193  0.093781   0.048067  0.51443

All              In VOC           Out of VOC          
            f1 micro  f1 macro  f1 micro  f1 macro   f1 micro  f1 macro
mean score  0.669837  0.516272  0.616576  0.246411   0.515327  0.489887
std         0.244483  0.061734  0.196617  0.152631    0.46726  0.024544

In [83]:
Evaluator(, test_concepts_without_conceptless)

['10036898',
 '10036898',
 'CONCEPT_LESS',
 '10036898',
 '10036898',
 '10063420',
 '10037430',
 '10042209',
 '10062519',
 '10029216',
 '10022998',
 '10011224',
 '10039101',
 '10005911',
 '10022437',
 '10022998',
 '10002855',
 '10042209',
 '10011224',
 '10028813',
 '10005911',
 '10039101',
 '10011224',
 '10037660',
 '10016766',
 '10013082',
 '10017367',
 '10020751',
 '10015150',
 '10013082',
 '10033616',
 '10000496',
 '10040007',
 '10039101',
 '10022437',
 '10062519',
 '10042209',
 '10042209',
 '10039101',
 '10039101',
 '10036898',
 '10033371',
 '10003239',
 '10003051',
 '10061218',
 '10023204',
 '10011224',
 '10039101',
 '10039101',
 '10039101',
 '10011224',
 '10011224',
 '10062519',
 '10029412',
 '10027783',
 '10062519',
 '10029216',
 '10068319',
 '10021425',
 '10012735',
 '10047700',
 '10011224',
 '10014551',
 '10022998',
 '10061284',
 '10022437',
 'CONCEPT_LESS',
 '10000060',
 '10061422',
 '10016766',
 '10039740',
 '10022998',
 '10010774',
 '10005750',
 '10028735',
 '10028735',
 '10

In [ ]:
#testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
net.eval()
model_answers=[]
real_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'][0])
        
    

In [26]:
f1_score(y_true=np.array([[1],[2],[3]]), y_pred=np.array([1,0,0]), average='micro')

0.3333333333333333

In [34]:
import numpy as np

a = np.array([1,2,3])
b = np.array([0,1, 3, 0, 0])

np.nonzero(~np.in1d(b, a))[0]

array([0, 3, 4])

In [38]:
b[np.array([0, 3, 4])]

array([0, 0, 0])

In [32]:
np.array(np.array([1,2,3]))

array([1, 2, 3])

In [12]:
net = torch.load('./Model_weights/rubert_50.pt')

In [13]:
from tqdm import tqdm
from sklearn.metrics import f1_score
import pandas as pd

testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)
net.eval()
model_answers=[]
real_answers=[]
with tqdm(testloader, unit="batch") as eval_process:
    for data in eval_process:

        inputs = data['tokenized_phrases']

        with torch.no_grad():
            outputs_dict = net(inputs)
            pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


        model_answers.append(pred_meddra_code)
        real_answers.append(data['label_codes'])
        
    f1 = f1_score(real_answers, model_answers, average='micro')
    print('%s micro'%f1)
    f1 = f1_score(real_answers, model_answers, average='macro')
    print('%s macro'%f1)
            


100%|██████████| 3787/3787 [00:41<00:00, 91.47batch/s] 


0.76234486400845 micro
0.2934589715874881 macro


---

In [14]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score



#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if USE_CUDA:
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scheduler.step()
                #scheduler.step()
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
                scheduler.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    f1 = f1_score(real_answers, model_answers, average='micro')
    if f1>best_f1:
        best_f1=f1
        torch.save(net,'./Model_weights/xlmrobertasag_best_cycliclr_%s.pt'%epoch)
    if epoch==9:
        torch.save(net,'./Model_weights/xlmrobertasag_cycliclr_%s.pt'%epoch)
    print(f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/xlmrobertasag_cycliclr_%s.pt'%epoch)

100%|██████████| 3787/3787 [01:22<00:00, 45.94batch/s]


0.28201742804330604


100%|██████████| 3787/3787 [01:14<00:00, 50.50batch/s]


0.4415104304198574


100%|██████████| 3787/3787 [01:16<00:00, 49.60batch/s]


...

0.7219434908898863


100%|██████████| 3787/3787 [01:15<00:00, 50.03batch/s]


0.7222075521520993


100%|██████████| 3787/3787 [01:14<00:00, 50.98batch/s]


0.7280168999207816
Finished Training


<h3>Процесс обучения и рассчета точностей xlmrobertasag</h3>

In [14]:
from tqdm import trange
from tqdm import tqdm
from sklearn.metrics import classification_report, f1_score

#Гиперпараметры
batch_size=16
epochs = 50
    
trainloader = torch.utils.data.DataLoader(ds_train, batch_size=batch_size,
                                          shuffle=False, num_workers=0)
testloader = torch.utils.data.DataLoader(ds_test, batch_size=1, shuffle=False)

net.train()
initial_loss = None
best_f1=0
for epoch in range(1, epochs+1):
    #обучение модели в эпохе
    net.train()
    with tqdm(trainloader, unit="batch") as tepoch:
        for data in tepoch:

            tepoch.set_description(f"Epoch {epoch}")

            inputs = data['tokenized_phrases']
            labels = data['one_hot_labels']

            optimizer.zero_grad()
            if device=='cuda':
                with torch.cuda.amp.autocast():
                    outputs = net(inputs)['output']
                    loss = criterion(outputs, labels)
                scaler.scale(loss).backward()
                scaler.step(optimizer)
                scaler.update()
            else:
                outputs = net(inputs)['output']
                loss = criterion(outputs, labels)
                loss.backward()
                optimizer.step()
            if initial_loss is None:
                initial_loss = loss.item()
            tepoch.set_postfix(loss_decrease = str(initial_loss/loss.item()))
    #расчет точностей модели после эпохи
    net.eval()
    model_answers=[]
    real_answers=[]
    with tqdm(testloader, unit="batch") as eval_process:
        for data in eval_process:

            inputs = data['tokenized_phrases']

            with torch.no_grad():
                outputs_dict = net(inputs)
                pred_meddra_code = CV_test_without_conceptless.meddra_codes[outputs_dict['output'].argmax()]


            model_answers.append(pred_meddra_code)
            real_answers.append(data['label_codes'])
    f1 = f1_score(real_answers, model_answers, average='micro')
    if f1>best_f1:
        best_f1=f1
        torch.save(net,'./Model_weights/xlmroberta_best_%s.pt'%epoch)
    if epoch==9:
        torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)
    print(f1)
            
print('Finished Training')
torch.save(net,'./Model_weights/xlmroberta_%s.pt'%epoch)

100%|██████████| 3787/3787 [01:16<00:00, 49.41batch/s]


0.2207552152099287


100%|██████████| 3787/3787 [01:21<00:00, 46.64batch/s]


0.39820438341695275


100%|██████████| 3787/3787 [01:17<00:00, 48.72batch/s]


...

0.7127013467124372


100%|██████████| 3787/3787 [01:15<00:00, 50.41batch/s]


0.7060998151571165


100%|██████████| 3787/3787 [01:16<00:00, 49.75batch/s]


0.7097966728280961
Finished Training


Инференс (подать фразу в сетку и посмотреть, какой концепт она выдаст)

Чтобы сетка выдавала отсутствие концепта на несвязанные с медрой фразы, раскомментируйте строчку с label_concepless_tensors

In [60]:
from transformers import AutoTokenizer

phrase = 'Disease'
tokenizer = AutoTokenizer.from_pretrained(transformer_model_path)
encoded_input = tokenizer([phrase], padding=True, truncation=True, return_tensors='pt')

with torch.no_grad():
    outputs_dict = net(inputs)
    #outputs_dict.label_concepless_tensors(score_treshold = 6.1977e-05)
    pred_meddra_code = CV.meddra_codes[outputs_dict['output'].argmax()]
    
print('phrase: %s'%phrase)
print('model output: %s'%CV.meddra_code_to_meddra_term[pred_meddra_code])

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


phrase: Disease
model output: Myalgia


<h2>Сохранение и загрузка модели</h2>

сохранение

In [63]:
torch.save(net, './trained_model_weights.pt')
torch.save(optimizer.state_dict(), './trained_model_opt.pt')

загрузка

In [64]:
the_model = torch.load('./trained_model_weights.pt')